# Notebook V
In this notebook we import an arbic dataset (from hagging face) and structure it to the format ("text", "emotion"). The challenge is that the observations in this dataset have multiple classes in the same row. Our job is to orgnize this dataset to be a multi-label dataset   

In [2]:
#Packages 
import pandas as pd
import numpy as np
from datasets import load_dataset 

d:\Users\chatbot_emotions_detection\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data 

In [4]:
#Importing data
data = load_dataset ( "sem_eval_2018_task_1", 'subtask5.arabic' )
data

DatasetDict({
    train: Dataset({
        features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 2278
    })
    test: Dataset({
        features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 1518
    })
    validation: Dataset({
        features: ['ID', 'Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
        num_rows: 585
    })
})

In [7]:
#Extracting train valiation test sets 
train = pd.DataFrame (data ['train'])
validation = pd.DataFrame (data ['validation'])
test = pd.DataFrame (data ['test'])

In [11]:
#concatinate the parts
data = pd.concat ( [train, validation, test] )

let's transform data from many columns representing emotions to one column "emotion"

In [74]:
#Liste des emotions 
emotions = list (data.columns[2:])
emotions
dataset = {'text': [], 'emotion': []}
for i in range (len (data)):
    ligne = data.iloc [i]
    num_of_emotions = 0
    list_of_emotions = []
    # Extracting num of emotions represented by a text
    for emotion in emotions :
        if ligne [emotion].item () == True:
            num_of_emotions += 1
            list_of_emotions.append (emotion)

    # transforming the ligne to one dictionnarie
    for j in range (num_of_emotions, ):
        dataset ["text"].append (ligne["Tweet"])
        dataset ['emotion'].append (list_of_emotions[j]) 
dataset

{'text': ['ظلينا نتكلم ساعات ساعات رتبت فيها نفسي وبكيت فيها مخاوفي مازلت أحس بشعور غريب وكأني رحلت من البيت للتو وهذا الشعور أقدر أتحمله',
  'ظلينا نتكلم ساعات ساعات رتبت فيها نفسي وبكيت فيها مخاوفي مازلت أحس بشعور غريب وكأني رحلت من البيت للتو وهذا الشعور أقدر أتحمله',
  'ظلينا نتكلم ساعات ساعات رتبت فيها نفسي وبكيت فيها مخاوفي مازلت أحس بشعور غريب وكأني رحلت من البيت للتو وهذا الشعور أقدر أتحمله',
  'ظلينا نتكلم ساعات ساعات رتبت فيها نفسي وبكيت فيها مخاوفي مازلت أحس بشعور غريب وكأني رحلت من البيت للتو وهذا الشعور أقدر أتحمله',
  'كل سنه وانتي بخير ياقلبي وكل سنه وانتي سعيده يارب 😍 ❤️ 🎈 🎈',
  'كل سنه وانتي بخير ياقلبي وكل سنه وانتي سعيده يارب 😍 ❤️ 🎈 🎈',
  'كل سنه وانتي بخير ياقلبي وكل سنه وانتي سعيده يارب 😍 ❤️ 🎈 🎈',
  'البسطاء يمتلكون أرواح نادره جداتجدهم بمظهر متواضع لكنهم بمحتوى باهض وراقي ينعمون بهدوء النفس لأنهم يمتلكون كنزالقناعة #سنام_المجد_للدعم',
  'البسطاء يمتلكون أرواح نادره جداتجدهم بمظهر متواضع لكنهم بمحتوى باهض وراقي ينعمون بهدوء النفس لأنهم يمتلكون كنزالقناعة #سنام_المج

In [82]:
#To dataframe 
dataframe = pd.DataFrame (dataset)

In [84]:
#Doing the emotional mapping, emotion to their original form
emotion_mapping = {'sadness':'sad', 'Sad': 'sad', 'pessimism': 'sad',
                    'joy': 'happy', 'happiness': 'happy', 'fun': 'happy', 'Laughing': 'happy', 'Happy': 'happy', 'love': 'happy', 'enthusiasm': 'happy', 'relief': 'happy', 'optimism': 'happy',
                    'none': 'neutral', 'Neutral': 'neutral', 'boredom': 'neutral', 'empty': 'neutral', 'sympathy': 'neutral', 'Wondering': 'neutral',
                    'Angry': 'angry', 'anger': 'angry', 'hate': 'angry', 'disgust': 'angry',
                    'Surprised': 'surprised', 'surprise': 'surprised',
                    'worry': 'fear', 'fear': 'fear'}
#Remaining emotions : ['sad', 'happy', 'surprised', 'fear', 'angry', neutral]
dataframe['emotion'] = dataframe['emotion'].replace (emotion_mapping)

In [91]:
#anticipation and trust emotions must be deleted
rows_to_delete = dataframe[ dataframe ['emotion'].isin (['anticipation', 'trust' ]) ].index
dataframe.drop ( index = rows_to_delete, inplace = True )

In [96]:
#Deleting duplicates 
dataframe = dataframe.drop_duplicates()

In [100]:
#Checking null values
dataframe.isna().sum()

text       0
emotion    0
dtype: int64

In [98]:
#final data
dataframe

,text,emotion
0,ظلينا نتكلم ساعات ساعات رتبت فيها نفسي وبكيت ف...,angry
1,ظلينا نتكلم ساعات ساعات رتبت فيها نفسي وبكيت ف...,fear
2,ظلينا نتكلم ساعات ساعات رتبت فيها نفسي وبكيت ف...,sad
4,كل سنه وانتي بخير ياقلبي وكل سنه وانتي سعيده ي...,happy
7,البسطاء يمتلكون أرواح نادره جداتجدهم بمظهر متو...,happy
...,...,...
10097,من الاصوات انا خذني و انا بزعل و انا حتى الزعل...,sad
10098,@imazizz ايش الحلو فيه دام بالاخير نهايه محزنه,sad
10100,الحزام يلتف على خصره كما تلتف أفعى جائعة على ف...,angry
10101,الحزام يلتف على خصره كما تلتف أفعى جائعة على ف...,fear


In [101]:
#Saving the dataframe 
SAVING_PATH = "D:/Users/chatbot_emotions_detection/Datasets/emtions_detection_datasets/text_emotion_arb_02.xlsx"
dataframe.to_excel (SAVING_PATH)